# TF  jensen-shannon diverse


__Author:__ Jingjing WANG

__Date:__ Dec. 2021

__Six Species:__ Xenopus

In [1]:
import time
from datetime import datetime
now=datetime.now()
print(now)

2022-01-04 20:56:18.844993


In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import os
import glob
import math

In [2]:
os.chdir("/media/ggj/ggjlab2/LiaoYuan/github_scripts/figure4/TF_expr_data/")

In [4]:
TPMlists=glob.glob("*_TF_expr.csv.gz")
TPMlists

['St54_TF_expr.csv.gz',
 'St66_TF_expr.csv.gz',
 'St59_TF_expr.csv.gz',
 'St48_TF_expr.csv.gz']

In [8]:
FOLDER='/media/ggj/ggjlab2/LiaoYuan/github_scripts/figure4/TF_expr_data/'
FOLDER1='/media/ggj/ggjlab2/LiaoYuan/github_scripts/figure4/phenotype_data/'
FOLDER2='/media/ggj/ggjlab2/LiaoYuan/github_scripts/figure4/bionary_data/'
RESULT_FOLDER='/media/ggj/ggjlab2/LiaoYuan/github_scripts/figure4/result/'

In [9]:
for TPMlist in TPMlists:
    TPMlist1=TPMlist.split("_TF")
    DATASET_ID=TPMlist1[0]
    print(DATASET_ID)

St54
St66
St59
St48


In [10]:
for TPMlist in TPMlists:
    TPMlist1=TPMlist.split("_TF")
    DATASET_ID=TPMlist1[0]
    
    Adata_File = os.path.join(FOLDER, '{}_TF_expr.csv.gz'.format(DATASET_ID))
    Meta_File = os.path.join(FOLDER1, '{}_TF_expr_pheno.csv.gz'.format(DATASET_ID))
    Meta_File2 = os.path.join(FOLDER2, '{}_TF_expr_bionary.txt.gz'.format(DATASET_ID))
    OUT_File1 = os.path.join(RESULT_FOLDER, '{}_TF_JSD.out'.format(DATASET_ID))
    OUT_File2 = os.path.join(RESULT_FOLDER, '{}_TF_JSDR.out'.format(DATASET_ID))
    
    adata_SC = sc.read_csv(Adata_File).T
    sc.pp.normalize_total(adata_SC, target_sum=1e4)
    #adata_SC = sc.read_csv(Adata_File)
    adata_SC_df = adata_SC.to_df().T
    adata_SC_df.columns = adata_SC_df.columns.astype("str")
    adata_SC_df.index = adata_SC_df.index.astype("str")
    #adata_SC_df.shape
    meta_SC = pd.read_csv(Meta_File,header=0,index_col="Cell")
    cell_index1 = adata_SC_df.columns.intersection(meta_SC.index)
    #cell_index1.shape[0]
    adata_SC_df_use = adata_SC_df
    meta_SC2 = pd.read_table(Meta_File2,sep="\t")
    Input1 = adata_SC_df_use # p matrix
    Input2 = meta_SC2 # q matrix
    Input1 = Input1.div(Input1.sum(axis=1),axis='rows')
    Input2 = Input2.div(Input2.sum(axis=1),axis='rows')
    JSD_list = []
    JSDR_list = []
    
    for i in range(Input1.shape[0]):
        for j in range(Input2.shape[0]):
            p = np.asarray(Input1.iloc[i,:]) + np.spacing(1)
            q = np.asarray(Input2.iloc[j,:]) + np.spacing(1)
            M=(p+q)/2 + np.spacing(1)   
            jsd_value=0.5*np.sum(p*np.log2(p/M))+0.5*np.sum(q*np.log2(q/M))
            jsd_r = 1- math.sqrt(jsd_value)
            JSD_list.append([Input1.index[i],Input2.index[j],jsd_value])
            JSDR_list.append([Input1.index[i],Input2.index[j],jsd_r])
        
    JSD_pd = pd.DataFrame(JSD_list,columns=['Cell','Celltype','JSD'])
    JSDR_pd = pd.DataFrame(JSDR_list,columns=['Cell','Celltype','JSDR'])
    JSD_pd.to_csv(OUT_File1,index = 0,sep="\t")
    JSDR_pd.to_csv(OUT_File2,index = 0,sep="\t")

In [22]:
#DONE